In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import chromedriver_autoinstaller
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup
import re
import json

# --크롬창을 숨기고 실행-- driver에 options를 추가해주면된다
# options = webdriver.ChromeOptions()
# options.add_argument('headless')

url = 'https://map.naver.com/v5/search'
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)  # 드라이버 경로
# driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)
key_word = '마포구 치킨'  # 검색어

C:\Users\Heejun\AppData\Local\Temp/ipykernel_10912/216879519.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path)  # 드라이버 경로


In [2]:
# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

In [3]:
# css를 찾을때 까지 10초 대기
time_wait(10, 'div.input_box > input.input_search')

# 검색창 찾기
search = driver.find_element_by_css_selector('div.input_box > input.input_search')
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

res = driver.page_source  # 페이지 소스 가져오기
soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다

sleep(1)

C:\Users\Heejun\AppData\Local\Temp/ipykernel_10912/662488279.py:5: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  search = driver.find_element_by_css_selector('div.input_box > input.input_search')


In [4]:
# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경
    res
    soup


# 페이지 다운
def page_down(num):
    body = driver.find_element_by_css_selector('body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

In [5]:
# frame 변경
switch_frame('searchIframe')
page_down(40)
sleep(5)

# 매장 리스트
store_list = driver.find_elements_by_css_selector('._1EKsQ')
# 페이지 리스트
next_btn = driver.find_elements_by_css_selector('._2ky45 > a')

# dictionary 생성
store_dict = {'매장정보': []}
# 시작시간
start = time.time()
print('[크롤링 시작...]')

C:\Users\Heejun\AppData\Local\Temp/ipykernel_10912/3960065300.py:11: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  body = driver.find_element_by_css_selector('body')


[크롤링 시작...]


C:\Users\Heejun\AppData\Local\Temp/ipykernel_10912/1686672749.py:7: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  store_list = driver.find_elements_by_css_selector('._1EKsQ')
C:\Users\Heejun\AppData\Local\Temp/ipykernel_10912/1686672749.py:9: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  next_btn = driver.find_elements_by_css_selector('._2ky45 > a')


In [ ]:
# 크롤링 (페이지 리스트 만큼)
for btn in range(len(next_btn))[1:]:  # next_btn[0] = 이전 페이지 버튼 무시 -> [1]부터 시작
    store_list
    for data in range(len(store_list)):  # 매장 리스트 만큼
        try:
            page = driver.find_elements_by_css_selector('.OXiLu')
            page[data].click()
            sleep(4)
        except:
            pass
        try:
            # 상세 페이지로 이동
            switch_frame('entryIframe')
            time_wait(5, '._3XamX')
            # 스크롤을 맨밑으로 1초간격으로 내린다.
            for down in range(3):
                sleep(1)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # -----매장명 가져오기-----
            store_name = driver.find_element_by_css_selector('._3XamX').text

            # -----평점-----
            try:
                store_rating_list = driver.find_element_by_css_selector('._1A8_M').text
                store_rating = re.sub('별점', '', store_rating_list).replace('\n', '')  # 별점이라는 단어 제거
            except:
                pass
            print(store_rating)

            # -----주소(위치)-----
            try:
                store_addr_list = driver.find_elements_by_css_selector('._1aj6-')
                for i in store_addr_list:
                    store_addr = i.find_element_by_css_selector('._1h3B_').text
            except:
                pass
            print(store_addr)
            # -----전화번호 가져오기-----
            try:
                store_tel = driver.find_element_by_css_selector('._3ZA0S').text
            except:
                pass

            print(store_tel)

            # -----메뉴-----
            try:
                menu_list = driver.find_elements_by_css_selector('._1XxR5')
                menu_name = []  # 메뉴이름
                menu_price = []  # 메뉴가격

                for i in menu_list:
                    name = i.find_element_by_css_selector('._1q3GD').text
                    menu_name.append(name)

                    price = i.find_element_by_css_selector('._3IAAc').text  # 텍스트를 넣을 변수 생성
                    price = re.sub('원\d{2},\d{3}', '', price)  # 할인 전 가격 제거  # 재정의
                    menu_price.append(price)  # 추가

            except:
                pass

            # -----메뉴2 (다른 태그의 메뉴)-----
            try:
                menu_list_two = driver.find_elements_by_css_selector('._20R25')
                menu_name_two = []
                menu_price_two = []

                for i in menu_list_two:
                    name_two = i.find_element_by_css_selector('._2E0Gk').text
                    name_two = re.sub('\n사진|\n대표', '', name_two)

                    price_two = i.find_element_by_css_selector('._3GJcI').text

                    menu_name_two.append(name_two)
                    menu_price_two.append(price_two)

            except:
                pass

            # 메뉴 리스트 합체
            menus = menu_name + menu_name_two
            prices = menu_price + menu_price_two

            print(menus)
            print(prices)



            # ---- dict에 데이터 집어넣기----
            dict_temp = {
                'name': store_name,
                'tel': store_tel,
                'star': store_rating,
                'addr': store_addr,
                'menu': menus,
                'price': prices,
            }

            store_dict['매장정보'].append(dict_temp)

            print(f'{store_name} ...완료')
            switch_frame('searchIframe')
            sleep(1)

        except:
            print('ERROR!' * 3)
            switch_frame('searchIframe')
            sleep(1)

    # 다음 페이지 버튼
    if page[-1]:  # 마지막 매장일 경우 다음버튼 클릭
        next_btn[-1].click()
        sleep(2)
    else:
        print('페이지 인식 못함')
        break

print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
driver.quit()  # 작업이 끝나면 창을닫는다.

In [9]:
# json 파일로 저장
file_path = ""
with open('./store_data.json', 'w', encoding='utf-8') as f:
    json.dump(store_dict, f, indent=4, ensure_ascii=False)

In [11]:
import pandas as pd
df = pd.DataFrame(store_dict)